<br>

## Initial Setup

---

In [7]:
# Import the libraries
import os

In [8]:
# Change the path to root directory
os.chdir('..') 

# Print the current working directory
os.getcwd()  

'/home/predator/Desktop/code base/enigma-code'

In [9]:
# Import the local modules
from src import data

<br>

## Load Dataset

---

In this section, we will load the dataset from various sources.

In [10]:
data.load_dataset()

True

<br>

## Fine-Tune Model

---

In this section, we will fine-tune the base model using the chat dataset.

<br>

## Vectorstore

---

Transform the knowledge base dataset into embedding vectors and save them in the vector store for use in the RAG process.

<br>

## Agentic Tools

---

<br>

## Prompt Engineering

---